<a href="https://colab.research.google.com/github/Ricardomanuel1/Maestria_Ciencia_de_Datos/blob/main/MACHINE%20LEARNING%20Y%20DEEP%20LEARNING/2_Fine_tuningSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Finetuning con Transformers (Sentiment Analysis)**

Finetuning implica entrenar un modelo pero un entrenamiento mas ligero no desde cero, es sobre los pesos de un modelo pre entrenado.

Basado en: https://huggingface.co/docs/transformers/tasks/sequence_classification

Temas:
- Fine-tuning de DistilBERT en el conjunto de datos de IMDb para determinar si la opinión de una película es positiva o negativa.
- Utilizar el modelo afinado para realizar inferencias.

Librerias

In [1]:
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import os
os._exit(00) # reinicia kernel solo porque se usa colab

In [1]:
!pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

**Carga de Dataset IMDB**

In [4]:
from datasets import load_dataset#datos abiertos IMDB

imdb = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Ver un ejemplo:

In [5]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [7]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Hay dos campos en este conjunto de datos:

- text: el texto de la reseña de la película.
- label: un valor que es 0 para una reseña negativa o 1 para una reseña positiva.

**Preprocesamiento**

El siguiente paso es tokenizar el texto en un formato legible por el modelo.

- Es importante cargar el mismo tokenizador con el que se entrenó un modelo para garantizar palabras tokenizadas de manera adecuada.

- Cargue el tokenizador DistilBERT con AutoTokenizer porque eventualmente entrenaremos un clasificador usando un modelo DistilBERT preentrenado:

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Ahora que ha instanciado un tokenizador, cree una función que tokenizará el texto. También debe truncar secuencias más largas en el texto para que no superen la longitud de entrada máxima del modelo:

In [8]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

Use la función de Datasets map 🤗 para aplicar la función de preprocesamiento a todo el conjunto de datos. También puede configurar ```batched=True``` para aplicar la función de preprocesamiento a varios elementos del conjunto de datos a la vez, para un preprocesamiento más rápido:

In [9]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)#TOKEN LUEGO A NUMEROS LUEGO EN UN VECTOR DE NUMEROS

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Por último, rellena (padding) el texto para que tenga una longitud uniforme. Si bien es posible rellenar el texto en la función tokenizadora configurando padding=True, es más eficiente rellenar el texto solo con la longitud del elemento más largo de su batch. Esto se conoce como relleno dinámico. Puede hacer esto con la función DataCollatorWithPadding:


In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**Evaluate**

- Incluir una métrica durante el entrenamiento suele ser útil para evaluar el rendimiento de su modelo. Puede cargar rápidamente un método de evaluación con la biblioteca 🤗 Evaluate.

- Para esta tarea, cargue la métrica de accuracy:

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

Luego cree una función que pase sus predicciones y etiquetas para calcular el accuracy:

In [12]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

**Clases**

Antes de comenzar a entrenar su modelo, cree un mapa de los identificadores esperados para las clases o etiquetas con id2label y label2id:

In [13]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

**Fine tuning con Trainer**

Ahora cargue su modelo con la clase ```AutoModelForSequenceClassification``` junto con la cantidad de etiquetas esperadas y los mapeos de las clases:

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)#CARGA LOS PESOS PERO SE ALTERA EL NUMERO DE CLASES A 2

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


En este punto, solo quedan tres pasos:

1. Defina sus hiperparámetros de entrenamiento en `TrainingArguments`. El único parámetro requerido es `output_dir` que especifica dónde guardar su modelo. Al final de cada época, el Trainer evaluará el accuracy  y guardará el checkpoint del entrenamiento.
2. Pase los argumentos de entrenamiento a un `Trainer` junto con el modelo, el conjunto de datos, el tokenizador y el data collator.
3. Llame a `Trainer.train()` para ajustar tu modelo.

In [16]:
training_args = TrainingArguments(
    output_dir="mi_modelo",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1, # era 2 por cuestión de tiempo
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    #push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
#data_collator=data_collator Se completa con ceros

Entrenamiento

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.216700,0.193472,0.926920


TrainOutput(global_step=1563, training_loss=0.25878217932663894, metrics={'train_runtime': 1530.2105, 'train_samples_per_second': 16.338, 'train_steps_per_second': 1.021, 'total_flos': 3280166004732288.0, 'train_loss': 0.25878217932663894, 'epoch': 1.0})

**Inferencia**

Prueba con un texto externo

In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

La forma más sencilla de probar su modelo ajustado para inferencia es usarlo en un `pipeline()`. Cree una instancia de `pipeline`
 el análisis de opiniones con su modelo y pásele su texto:

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="mi_modelo/checkpoint-1563")
classifier(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9923667311668396}]

También puedes replicar manualmente los resultados del `pipeline`:

Tokenice el texto y devuelva los tensores de PyTorch:

In [ ]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("mi_modelo/checkpoint-1563")
inputs = tokenizer(text, return_tensors="pt")

Pase sus entradas al modelo y devuelva los logits:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("mi_modelo/checkpoint-1563")
with torch.no_grad():
    logits = model(**inputs).logits

Obtenga la clase con la mayor probabilidad y use el mapeo id2label del modelo para convertirla en una etiqueta de texto:

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'POSITIVE'